# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
#import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\301518\AppData\Local\Continuum\anaconda3\envs\Cafetalk_1\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [2]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
df_customer.application_date = pd.to_datetime(df_customer.application_date)
df_customer[["customer_id", "application_date"]].head(10)
# pd.to_datetime() が上手くできていない

,customer_id,application_date
0,CS021313000114,1970-01-01 00:00:00.020150905
1,CS037613000071,1970-01-01 00:00:00.020150414
2,CS031415000172,1970-01-01 00:00:00.020150529
3,CS028811000001,1970-01-01 00:00:00.020160115
4,CS001215000145,1970-01-01 00:00:00.020170605
5,CS020401000016,1970-01-01 00:00:00.020150225
6,CS015414000103,1970-01-01 00:00:00.020150722
7,CS029403000008,1970-01-01 00:00:00.020150515
8,CS015804000004,1970-01-01 00:00:00.020150607
9,CS033513000180,1970-01-01 00:00:00.020150728


In [4]:
# 原因解明
print(pd.to_datetime(20200824))
print(pd.to_datetime("20200824"))
# pd.to_datetime() の引数はstr型でないといけない

1970-01-01 00:00:00.020200824
2020-08-24 00:00:00


In [5]:
df_customer = pd.read_csv("./data/customer.csv")
df_customer.info

<bound method DataFrame.info of           customer_id customer_name  gender_cd gender   birth_day  age  \
0      CS021313000114        大野 あや子          1     女性  1981-04-29   37   
1      CS037613000071         六角 雅彦          9     不明  1952-04-01   66   
2      CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42   
3      CS028811000001        堀井 かおり          1     女性  1933-03-27   86   
4      CS001215000145         田崎 美紀          1     女性  1995-03-29   24   
...               ...           ...        ...    ...         ...  ...   
21966  CS002512000474         市村 夏希          1     女性  1959-10-12   59   
21967  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   
21968  CS012403000043          堀越 明          0     男性  1972-12-16   46   
21969  CS033512000184         池谷 華子          1     女性  1964-06-05   54   
21970  CS009213000022          稲垣 咲          1     女性  1996-08-16   22   

      postal_cd                  address application_store_cd  \
0      259-111

In [6]:
# まずapplication_date をstring型に変更する必要あり
df_customer.application_date = df_customer.application_date.astype("str")
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21971 entries, 0 to 21970
Data columns (total 11 columns):
customer_id             21971 non-null object
customer_name           21971 non-null object
gender_cd               21971 non-null int64
gender                  21971 non-null object
birth_day               21971 non-null object
age                     21971 non-null int64
postal_cd               21971 non-null object
address                 21971 non-null object
application_store_cd    21971 non-null object
application_date        21971 non-null object
status_cd               21971 non-null object
dtypes: int64(2), object(9)
memory usage: 1.8+ MB


In [7]:
df_customer.application_date = pd.to_datetime(df_customer.application_date)
df_customer[["customer_id", "application_date"]].head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [8]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [9]:
df_receipt.sales_ymd = pd.to_datetime(df_receipt.sales_ymd.astype("str"))
df_receipt[["receipt_no", "sales_ymd"]].head(10)

,receipt_no,sales_ymd
0,112,2018-11-03
1,1132,2018-11-18
2,1102,2017-07-12
3,1132,2019-02-05
4,1102,2018-08-21
5,1112,2019-06-05
6,1102,2018-12-05
7,1102,2019-09-22
8,1112,2017-05-04
9,1102,2019-10-10


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [10]:
df_receipt = pd.read_csv("./data/receipt.csv")

In [11]:
# https://deepage.net/features/pandas-to-datetime.html#unix%E6%99%82%E9%96%93%E3%81%8B%E3%82%89%E3%81%AE%E5%A4%89%E6%8F%9B
# 今度は世界標準時(UTC)で1970年1月1日 0時0分からの経過時間を表示するUNIX時間を変換します。 
# 試しに2018年9月20日13時25分00秒(日本時間)のUNIX時間である”1537417500”を変換してみます。
# ここでの表記は秒数なので、 unit='s'として表記を秒単位にします

df_receipt.sales_epoch = pd.to_datetime(df_receipt.sales_epoch.astype("str"), unit="s")
df_receipt[["receipt_no", "receipt_sub_no", "sales_epoch"]].head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009-11-03
1,1132,2,2009-11-18
2,1102,1,2008-07-12
3,1132,1,2010-02-05
4,1102,2,2009-08-21
5,1112,1,2010-06-05
6,1102,2,2009-12-05
7,1102,1,2010-09-22
8,1112,2,2008-05-04
9,1102,1,2010-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [12]:
df_receipt = pd.read_csv("./data/receipt.csv")

In [13]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.year.html
# pd の datetime Series のメンバー "dt" のさらにメンバ "year" を取り出す

df_receipt.sales_epoch = pd.to_datetime(df_receipt.sales_epoch.astype("str"), unit="s").dt.year
df_receipt[["receipt_no", "receipt_sub_no", "sales_epoch"]].head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009
1,1132,2,2009
2,1102,1,2008
3,1132,1,2010
4,1102,2,2009
5,1112,1,2010
6,1102,2,2009
7,1102,1,2010
8,1112,2,2008
9,1102,1,2010


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [14]:
df_receipt = pd.read_csv("./data/receipt.csv")

In [15]:
df_receipt.sales_epoch = pd.to_datetime(df_receipt.sales_epoch.astype("str"), unit="s").dt.month
df_receipt[["receipt_no", "receipt_sub_no", "sales_epoch"]].head(10)
# これでは"月"は0埋め2桁で取り出せていない

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,7
3,1132,1,2
4,1102,2,8
5,1112,1,6
6,1102,2,12
7,1102,1,9
8,1112,2,5
9,1102,1,10


In [16]:
df_receipt = pd.read_csv("./data/receipt.csv")

In [18]:
# https://www.javadrive.jp/ruby/date_class/index5.html

df_receipt.sales_epoch = pd.to_datetime(df_receipt.sales_epoch.astype("str"), unit="s").dt.strftime('%m')
df_receipt[["receipt_no", "receipt_sub_no", "sales_epoch"]].head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10
